In [2]:
# LLM model call (in LangChain)

#from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

#llm = OpenAI(model_name="gpt-3.5-turbo-1106")
chat = ChatOpenAI()

#a = llm.predict("How many planets are there?")
b = chat.predict("How many planets are there?")

# ChatOpenAI is cheaper and more chatable than OpenAI. so don't use OpenAI anymore. (+ also OpenAI is due-dated) 
b

'There are eight recognized planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.'

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.2
)

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}. Also, what is your name?"
    )
prompt = template.format(country_a="Mexico", country_b="Thailand")


chat.predict(prompt)




'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles). My name is Assistant.'

In [14]:
# from langchain.schema import HumanMessage, AIMessage, SystemMessage (if you use ChatPromptTemplate, you don't need this library)
# Prompt Templates : use placeholder{} and exchage country, name, etc.. from message list
template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply only {lang}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?")
])
# SystemMessage is to setting for LLM

prompt = template.format_messages(
    lang="Korean",
    name="Kim Soo Hyun",
    country_a="Mexico",
    country_b="Thailand"
)

chat.predict_messages(prompt)

AIMessage(content='멕시코와 태국 사이의 거리는 대략 16,000km입니다. 제 이름은 김수현입니다.')

In [ ]:
# Output parser : We need this because sometimes you have to transform 'LLM Response' 
# we gonna make response 'text(string)' type to 'list'
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    # Outputparser should implment def 'parse'!
    def parse(self, text):
        items =  text.strip().split(",")
        return list(map(str.strip, items))
    

p = CommaOutputParser()
p.parse("Hello,how,    are,you")



['Hello', 'how', 'are', 'you']

In [ ]:
# We already import 'AIMessagefrom langchain.prompts ..' so pass.
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowerecase. DO NOT reply with anything else."),
    ("human", "{question}")
])

prompt = template.format_messages(
    max_items = 10,
    question = "What are the colors?"
)

result = chat.predict_messages(prompt)

# Using Outputparser
p.parse(result.content)

# Ok. cool. But this Code is toooo long and lot a things to do. 
# Let's change this code with 'LangChain Expression Language'!


['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'black',
 'white',
 'pink',
 'brown']

In [ ]:
# LangChain Expression Language
# template, prompt and predict. These thigs can exchange this. only one code can replace these three things.
# We only need 'ChatOpenAI (already done) / CommaOutputParser (already done) / ChatPromptTemplate (doing now)
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowerecase. DO NOT reply with anything else."),
    ("human", "{question}")
])

chain_one = template | chat | CommaOutputParser() #create 'chain' for LangChain
#chain_two = template2 | chat | outputparser
#all  = chain_one | chain_two | output..

chain_one.invoke({
    "max_items": 5,
    "question": "Wat are the pokemons?"
})

# LangChain library is amazing

['pikachu', 'bulbasaur', 'charmander', 'squirtle', 'jigglypuff']

In [ ]:
# Using LangChain from zero(=nothing)

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, 
                  streaming=True, 
                  callbacks=[StreamingStdOutCallbackHandler()]) # you can see strings(log..?) what LLM doing now

# chain + chain expression
chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. "
    "You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat

vegan_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipes vegetarian. "
    "You find alternative ingredients and explain their preparation. "
    "You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to recipe it."),
    ("human", "{recipe}")
])

vegan_chain = vegan_chef_prompt | chat


final_chain = {"recipe": chef_chain} | vegan_chain

final_chain.invoke({
    "cuisine": "indian"
})


Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic and popular dish - Chicken Tikka Masala. Here's a simple recipe for you to try at home:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 teaspoons ground cumin
- 2 teaspoons ground coriander
- 1 teaspoon turmeric
- 1 teaspoon paprika
- 1 teaspoon garam masala
- 1 teaspoon salt
- 1 tablespoon vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1 tablespoon grated ginger
- 1 can (14 oz) crushed tomatoes
- 1 cup heavy cream
- Fresh cilantro, chopped (for garnish)

Instructions:
1. In a bowl, mix together the yogurt, lemon juice, cumin, coriander, turmeric, paprika, garam masala, and salt. Add the chicken pieces and coat them well with the marinade. Cover and refrigerate for at least 1 hour, or overnight for best results.

2. Preheat the oven to 400°F (200°C). Place the marinated ch

AIMessageChunk(content="For a vegetarian version of Chicken Tikka Masala, you can easily substitute the chicken with a plant-based alternative such as tofu or paneer. Here's how you can adapt the recipe:\n\nIngredients:\n- 1 lb firm tofu or paneer, cut into bite-sized pieces\n- 1 cup plain yogurt (you can use dairy-free yogurt for a vegan version)\n- 2 tablespoons lemon juice\n- 2 teaspoons ground cumin\n- 2 teaspoons ground coriander\n- 1 teaspoon turmeric\n- 1 teaspoon paprika\n- 1 teaspoon garam masala\n- 1 teaspoon salt\n- 1 tablespoon vegetable oil\n- 1 onion, finely chopped\n- 3 cloves garlic, minced\n- 1 tablespoon grated ginger\n- 1 can (14 oz) crushed tomatoes\n- 1 cup coconut cream (or any other dairy-free cream alternative)\n- Fresh cilantro, chopped (for garnish)\n\nInstructions:\n1. Follow the same marinating process as the original recipe, but instead of chicken, marinate the tofu or paneer in the yogurt and spice mixture. Refrigerate for at least 1 hour.\n\n2. Bake the m

In [ ]:
# Model I/O : FewShotPromptTemplate (important!!)


In [ ]:
# Chat model vs LLM (both are in LangChain)